In [ ]:
#% pip install -q pyterrier_rag pyterrier_t5

In [3]:
import pyterrier_rag
import pyterrier as pt

In [7]:
import os
os.environ["IDA_LLM_API_KEY"] = "ida_aiLXPKeKHqQ5MLpjS1mSNjWtCz6Sc9Xyy8qbunhS"

In [8]:
bm25 = pt.Artifact.from_hf("pyterrier/vaswani.terrier").bm25()
textl = pt.text.get_text(pt.get_dataset("irds:vaswani"))

In [9]:
# you may need to set IDA_LLM_API_KEY for this to run
import os
backend = pyterrier_rag.OpenAIBackend("llama-3-8b-instruct", api_key=os.environ["IDA_LLM_API_KEY"], base_url="http://api.terrier.org/v1")
backend

OpenAIBackend('llama-3-8b-instruct')

In [10]:
backend.text_generator()

In [11]:
reader = pyterrier_rag.readers.Reader(backend)
reader

In [12]:
bm25 >> textl >> pyterrier_rag.prompt.Concatenator() >> reader

(TerrierRetr(BM25) >> <pyterrier.datasets._irds.IRDSTextLoader object at 0x367ce0140> >> <pyterrier_rag.prompt._context_aggregation.Concatenator object at 0x367ce09e0> >> <pyterrier_rag.readers._base.Reader object at 0x367ce10a0>)

In [15]:
flant5 = pyterrier_rag.Seq2SeqLMBackend("google/flan-t5-base")
flant5

Seq2SeqLMBackend('google/flan-t5-base')

In [16]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [17]:
ircot = pyterrier_rag.frameworks.IRCOT(bm25 >> textl >> monoT5, flant5)
ircot

monoT5: 0batches [00:00, ?batches/s]
monoT5: 0batches [00:00, ?batches/s]
monoT5: 0batches [00:00, ?batches/s]


In [18]:
ircotbad = pyterrier_rag.frameworks.IRCOT(bm25 >> monoT5, flant5)
ircotbad